In [18]:
%load_ext blackcellmagic

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [1]:
import pandas as pd
import vaex
from decouple import config

In [20]:
#pipenv uninstall pytables

In [21]:
df = vaex.open("million_9_song(1).hdf5")

In [22]:
si = df.song_identifier.evaluate()

In [23]:
annoy_index = df.index.evaluate()

In [24]:
song_info = list(zip(si,annoy_index))

In [25]:
song_info[0]

('dangerkids-hostage', 0)

## Push to Elastic Search

In [2]:
from elasticsearch import Elasticsearch 

In [54]:
es=Elasticsearch(config('ELASTIC_SEARCH_DOMAIN'))

In [55]:
es

<Elasticsearch([{'host': 'search-music-recommender-wwryojqvlkrgqk3a5xiqtubwzy.us-east-2.es.amazonaws.com', 'port': 443, 'use_ssl': True}])>

In [56]:
#es.indices.create(index='song-index', ignore=[400,404])

In [31]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [32]:
s1={"song-identifier":"dangerkids-hostage","annoy-id":1}
res = es.index(index='song-index', doc_type='song-doc',id=1,body=s1)

In [33]:
res

{'_index': 'song-index',
 '_type': 'song-doc',
 '_id': '1',
 '_version': 2,
 'result': 'updated',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 1,
 '_primary_term': 1}

In [34]:
def song_body(body_index_tuple):
    body = {
        "_index": "song-index",
        "_type": "song-doc",
        "_id": int(body_index_tuple[1]), 
        "song-identifier": body_index_tuple[0],
        "annoy-id": int(body_index_tuple[1]),
    }
    return body

In [35]:
import warnings
warnings.filterwarnings('ignore')

## Bulk upload
So far unsuccessful

In [36]:
from elasticsearch.helpers import bulk

In [37]:
def gendata():
    mywords = ['foo', 'bar', 'baz']
    for song in song_info:
        yield song_body(song)

bulk(es, gendata())

(1844757, [])

# Testing ES

In [3]:
from elasticsearch import Elasticsearch
es=Elasticsearch(config('ELASTIC_SEARCH_DOMAIN'))

In [4]:
 es.search(
        index="song-index",
        body={"query": {"match": {"song-identifier": "wu tang clang triumph"}}},
    )["hits"]["hits"]

INFO:MainThread:elasticsearch:GET https://search-music-recommender-wwryojqvlkrgqk3a5xiqtubwzy.us-east-2.es.amazonaws.com:443/song-index/_search [status:200 request:0.167s]


[{'_index': 'song-index',
  '_type': 'song-doc',
  '_id': '1655345',
  '_score': 28.365307,
  '_source': {'song-identifier': 'wu-tang-clan-triumph', 'annoy-id': 1655345}},
 {'_index': 'song-index',
  '_type': 'song-doc',
  '_id': '1494445',
  '_score': 22.786552,
  '_source': {'song-identifier': 'wu-tang-clan-wu-tang-master',
   'annoy-id': 1494445}},
 {'_index': 'song-index',
  '_type': 'song-doc',
  '_id': '988475',
  '_score': 22.450748,
  '_source': {'song-identifier': 'wu-tang-clan-wu-tang-reunion',
   'annoy-id': 988475}},
 {'_index': 'song-index',
  '_type': 'song-doc',
  '_id': '1657276',
  '_score': 21.441769,
  '_source': {'song-identifier': 'wu-tang-clan-wu-tang-7th-chamber',
   'annoy-id': 1657276}},
 {'_index': 'song-index',
  '_type': 'song-doc',
  '_id': '1494475',
  '_score': 21.034655,
  '_source': {'song-identifier': 'wu-tang-clan-enter-the-wu-tang',
   'annoy-id': 1494475}},
 {'_index': 'song-index',
  '_type': 'song-doc',
  '_id': '1494479',
  '_score': 21.034655,
 

In [5]:
q = [
    {
        "song-identifier": i["_source"]["song-identifier"],
        "annoy-id": i["_source"]["annoy-id"],
    }
    for i in es.search(
        index="song-index",
        body={"query": {"match": {"song-identifier": "wu tang clang triumph"}}},
    )["hits"]["hits"]
]

INFO:MainThread:elasticsearch:GET https://search-music-recommender-wwryojqvlkrgqk3a5xiqtubwzy.us-east-2.es.amazonaws.com:443/song-index/_search [status:200 request:0.042s]


In [6]:
q[0]['annoy-id']

1655345

In [7]:
from annoy import AnnoyIndex

In [8]:
b = AnnoyIndex(200)
b.load('../app/songs.ann')

True

In [9]:
z = b.get_nns_by_item(q[0]['annoy-id'], 10)

In [10]:
z

[1655345,
 1473340,
 1147912,
 1657793,
 1489749,
 1657252,
 1659510,
 1656813,
 1453730,
 192700]

In [11]:
es.get(index='song-index',doc_type='song-doc',id=192700)

INFO:MainThread:elasticsearch:GET https://search-music-recommender-wwryojqvlkrgqk3a5xiqtubwzy.us-east-2.es.amazonaws.com:443/song-index/song-doc/192700 [status:200 request:0.040s]


{'_index': 'song-index',
 '_type': 'song-doc',
 '_id': '192700',
 '_version': 1,
 '_seq_no': 38564,
 '_primary_term': 1,
 'found': True,
 '_source': {'song-identifier': 'melvin-bliss-synthetic-substitution',
  'annoy-id': 192700}}

In [12]:
es.get(index='song-index',doc_type='song-doc',id=q[5]['annoy-id'])

INFO:MainThread:elasticsearch:GET https://search-music-recommender-wwryojqvlkrgqk3a5xiqtubwzy.us-east-2.es.amazonaws.com:443/song-index/song-doc/1494479 [status:200 request:0.036s]


{'_index': 'song-index',
 '_type': 'song-doc',
 '_id': '1494479',
 '_version': 1,
 '_seq_no': 299123,
 '_primary_term': 1,
 'found': True,
 '_source': {'song-identifier': 'wu-tang-clan-wu-tang-forever-intro',
  'annoy-id': 1494479}}

In [24]:
def make_playlist(search):
    query = es.search(
        index="song-index", body={"query": {"match": {"song-identifier": search}}}
    )["hits"]["hits"][0]["_source"]["annoy-id"]
    query = b.get_nns_by_item(q[0]["annoy-id"], 10)
    playlist =  list(
        map(
            lambda x: es.get(index="song-index", doc_type="song-doc", id=x)["_source"][
                "song-identifier"
            ],
            query,
        )
    )
    playlist = [{"identifier": i, "url" : "https://genius.com/"+i+"-lyrics"} for i in playlist]
    return playlist

In [25]:
make_playlist("wu tang clan triumph")

INFO:MainThread:elasticsearch:GET https://search-music-recommender-wwryojqvlkrgqk3a5xiqtubwzy.us-east-2.es.amazonaws.com:443/song-index/_search [status:200 request:0.045s]
INFO:MainThread:elasticsearch:GET https://search-music-recommender-wwryojqvlkrgqk3a5xiqtubwzy.us-east-2.es.amazonaws.com:443/song-index/song-doc/1655345 [status:200 request:0.036s]
INFO:MainThread:elasticsearch:GET https://search-music-recommender-wwryojqvlkrgqk3a5xiqtubwzy.us-east-2.es.amazonaws.com:443/song-index/song-doc/1473340 [status:200 request:0.040s]
INFO:MainThread:elasticsearch:GET https://search-music-recommender-wwryojqvlkrgqk3a5xiqtubwzy.us-east-2.es.amazonaws.com:443/song-index/song-doc/1147912 [status:200 request:0.037s]
INFO:MainThread:elasticsearch:GET https://search-music-recommender-wwryojqvlkrgqk3a5xiqtubwzy.us-east-2.es.amazonaws.com:443/song-index/song-doc/1657793 [status:200 request:0.037s]
INFO:MainThread:elasticsearch:GET https://search-music-recommender-wwryojqvlkrgqk3a5xiqtubwzy.us-east-2.

[{'identifier': 'wu-tang-clan-triumph',
  'url': 'https://genius.com/wu-tang-clan-triumph-lyrics'},
 {'identifier': 'gang-of-four-hed-send-in-the-army',
  'url': 'https://genius.com/gang-of-four-hed-send-in-the-army-lyrics'},
 {'identifier': 'bing-crosby-deep-in-the-heart-of-texas',
  'url': 'https://genius.com/bing-crosby-deep-in-the-heart-of-texas-lyrics'},
 {'identifier': 'wu-tang-clan-wu-tang-clan-aint-nuthing-ta-fuck-wit',
  'url': 'https://genius.com/wu-tang-clan-wu-tang-clan-aint-nuthing-ta-fuck-wit-lyrics'},
 {'identifier': 'snoop-dogg-g-funk-intro',
  'url': 'https://genius.com/snoop-dogg-g-funk-intro-lyrics'},
 {'identifier': 'public-enemy-black-steel-in-the-hour-of-chaos',
  'url': 'https://genius.com/public-enemy-black-steel-in-the-hour-of-chaos-lyrics'},
 {'identifier': 'mobb-deep-animal-instinct',
  'url': 'https://genius.com/mobb-deep-animal-instinct-lyrics'},
 {'identifier': 'gza-4th-chamber',
  'url': 'https://genius.com/gza-4th-chamber-lyrics'},
 {'identifier': 'origi